In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 700 kB 5.1 MB/s 
     |████████████████████████████████| 5.8 MB 62.1 MB/s 
     |████████████████████████████████| 141 kB 74.3 MB/s 
     |████████████████████████████████| 596 kB 74.5 MB/s 
     |████████████████████████████████| 419 kB 60.0 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.9.1 which is inco

In [3]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F

In [37]:
pl.seed_everything(7)

Global seed set to 7


7

In [63]:
import torchvision
class Data(pl.LightningDataModule):
  def __init__(self, fold=0, input_height=320, input_width=320, train_batch_size=16, val_batch_size=16,
               n_classes=2):
    super().__init__()
    self.data_dir = "/content/drive/MyDrive/textline_segmentation_using_fcn/"
    self.weights = torchvision.models.segmentation.FCN_ResNet50_Weights.DEFAULT
    self.transform = self.weights.transforms()
    self.fold = fold
    self.input_height = input_height
    self.input_width = input_width
    self.train_batch_size = train_batch_size
    self.val_batch_size = val_batch_size
    self.n_classes = n_classes
    self.output_height = 520
    self.output_width = 520

  def prepare_data(self):
    pass

  def setup(self, stage=None):
    pass

  def train_dataloader(self):
    return DataLoader(self.train_set, self.train_batch_size)

  def val_dataloader(self):
    return DataLoader(self.val_set, self.val_batch_size)

  def test_dataloader(self):
    pass

  def visualize(self):
    pass

In [64]:
dm = Data()
dm.setup()

In [65]:
dm.train_set

<generator object imageSegmentationGenerator at 0x7ff85e95f5d0>

In [45]:
class LitFCN(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.weights = torchvision.models.segmentation.FCN_ResNet50_Weights.DEFAULT
    self.model = torchvision.models.segmentation.fcn_resnet50(weights=self.weights)
    self.model.eval()
    self.save_hyperparameters()

  def forward(self, x):
    x = self.model(x)
    return x

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    loss = F.cross_entropy(logits, y)
    self.log("train loss", loss)
    return loss

  def evaluate(self, batch, stage=None):
    x, y = batch
    logits = self.forward(x)
    loss = F.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, y)

    if stage:
      self.log(f"{stage}_loss", loss, prog_bar=True)
      self.log(f"{stage}_acc", acc, prog_bar=True)

  def validation_step(self, batch, batch_idx):
    self.evaluate(batch, stage="val")

  def test_step(self, batch, batch_idx):
    self.evaluate(batch, stage="test")

  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=0.001)

In [46]:
model = LitFCN()
trainer = pl.Trainer(max_epochs=1,
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=20)],
)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/lightning_logs

  | Name  | Type | Params
-------------------------------
0 | model | FCN  | 35.3 M
-------------------------------
35.3 M    Trainable params
0         Non-trainable params
35.3 M    Total params
141.289   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

TypeError: ignored